In [1]:
from project.dataset import project_data
import os
import numpy as np
from collections import defaultdict
import datetime
import pytz
import pandas as pd

ModuleNotFoundError: No module named 'project'

In [0]:
root_path = 'C:\Users\kyrie\Desktop\comp9417\project\StudentLife_Dataset'
#root_path = os.path.join(os.path.dirname(__file__), 'StudentLife_Dataset')
data = project_data(root_path=root_path)


In [0]:
stud_id = data.pre_student
flour = data.FlourishingScale()
panas_pos, panas_neg = data.PANAS()


In [0]:
counter = 0
for key in flour:
    counter += flour[key]
mean_flour = counter / len(flour)


In [0]:
counter = 0
for key in panas_pos:
    counter += panas_pos[key]
mean_panas_pos = counter / len(panas_pos)


In [0]:
counter = 0
for key in panas_neg:
    counter += panas_neg[key]
mean_panas_neg = counter / len(panas_neg)


In [0]:
y_flour, y_panas_pos, y_panas_neg = [], [], []
for s in stud_id:
    if flour[s] > mean_flour:
        y_flour.append(1)
    else:
        y_flour.append(0)
    if panas_pos[s] > mean_panas_pos:
        y_panas_pos.append(1)
    else:
        y_panas_pos.append(0)
    if panas_neg[s] > mean_panas_neg:
        y_panas_neg.append(1)
    else:
        y_panas_neg.append(0)
y_flour = np.array(y_flour)
y_panas_pos = np.array(y_panas_pos)
y_panas_neg = np.array(y_panas_neg)
    

In [0]:
conv_dura_day, conv_dura_eve, conv_dura_nig = data.conversation_dura()
conv_freq_day, conv_freq_eve, conv_freq_nig = data.conversation_freq()
co_location = data.co_location()
activity_day, activity_eve, activity_nig = data.activity()
distance_day, distance_eve, distance_nig = data.traveled_distance()
indoor_mobility_day, indoor_mobility_eve, indoor_mobility_nig = data.indoor_mobility()


In [0]:
#sleep_duration = data.sleep_duration()


In [0]:
# conversation duration (pre) 0.294 0.066
# conversation duration during evening (pre) 0.362 0.022
# number of co-locations (post) 0.324 0.050
x_flour = []
for s in stud_id:
    temp = []
    temp.append(conv_dura_day[s] + conv_dura_eve[s] + conv_dura_nig[s])
    temp.append(conv_dura_eve[s])
    temp.append(co_location[s])
    x_flour.append(temp)
x_flour = np.array(x_flour)


In [0]:
# sleep duration (pre) -0.360 0.025
# conversation frequency during day (pre) -0.403 0.010
# conversation frequency during evening (post) -0.345 0.034
# conversation duration during day (post) -0.328 0.044
# number of co-locations (post) -0.362 0.025
# activity duration for day (post) -0.326 0.049
# activity duration for evening (post) -0.464 0.004
# traveled distance (post) -0.338 0.044
# traveled distance for day (post) -0.336 0.042
# indoor mobility for day (post) -0.332 0.045
x_panas = []
for s in stud_id:
    temp = []
    #temp.append(sleep_duration[s])
    temp.append(conv_freq_day[s])
    temp.append(conv_freq_eve[s])
    temp.append(conv_dura_day[s])
    temp.append(conv_dura_eve[s])
    temp.append(co_location[s])
    temp.append(activity_day[s])
    temp.append(activity_eve[s])
    temp.append(distance_day[s] + distance_eve[s] + distance_nig[s])
    temp.append(distance_day[s])
    temp.append(indoor_mobility_day[s])
    x_panas.append(temp)
x_panas = np.array(x_panas)


In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt


In [0]:
print(x_flour.shape)
print(x_panas.shape)
print(y_flour.shape)
print(y_panas_pos.shape)
print(y_panas_neg.shape)
scaler = MinMaxScaler()
x_flour_scaled = scaler.fit_transform(x_flour)
x_panas_scaled = scaler.fit_transform(x_panas)


In [0]:
k_range = range(1, 31)
weight_options = ['uniform', 'distance']
param_grid = {'n_neighbors':k_range,'weights':weight_options}
knn = KNeighborsClassifier(n_neighbors=1)
grid = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='accuracy', cv=10)


In [0]:
grid.fit(x_flour_scaled, y_flour)
# print('网格搜索-度量记录：',grid.cv_results_)  
print('网格搜索-最佳度量值:',grid.best_score_)  
print('网格搜索-最佳参数：',grid.best_params_)  
print('网格搜索-最佳模型：',grid.best_estimator_) 


In [0]:
grid.fit(x_panas_scaled, y_panas_pos)
# print('网格搜索-度量记录：',grid.cv_results_)  
print('网格搜索-最佳度量值:',grid.best_score_)  
print('网格搜索-最佳参数：',grid.best_params_)  
print('网格搜索-最佳模型：',grid.best_estimator_) 


In [0]:
grid.fit(x_panas_scaled, y_panas_neg)
# print('网格搜索-度量记录：',grid.cv_results_)  
print('网格搜索-最佳度量值:',grid.best_score_)  
print('网格搜索-最佳参数：',grid.best_params_)  
print('网格搜索-最佳模型：',grid.best_estimator_) 

